In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0


In [2]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    vgg_input_tensor = graph.get_tensor_by_name(vgg_input_tensor_name)
    vgg_keep_prob_tensor = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    vgg_layer3_out_tensor = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    vgg_layer4_out_tensor = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    vgg_layer7_out_tensor = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)    
    
    return vgg_input_tensor, vgg_keep_prob_tensor, vgg_layer3_out_tensor, vgg_layer4_out_tensor, vgg_layer7_out_tensor
tests.test_load_vgg(load_vgg, tf)


def layers(vgg_layer3_encode, vgg_layer4_encode, vgg_layer7_encode, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    vgg_layer7_out = tf.layers.conv2d(vgg_layer7_encode, num_classes, 1, padding='same',
                                      kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer4_1 = tf.layers.conv2d_transpose(vgg_layer7_out, num_classes, 4, strides=(2,2), padding='same',
                                          kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    layer_4_2 = tf.layers.conv2d(vgg_layer4_encode, num_classes, 1, padding='same',
                                 kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    layer4_out = tf.add(layer4_1, layer_4_2)
    
    layer3_1 = tf.layers.conv2d_transpose(layer4_out, num_classes, 4, strides=(2,2), padding='same',
                                          kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    layer_3_2 = tf.layers.conv2d(vgg_layer3_encode, num_classes, 1, padding='same',
                                 kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    layer3_out = tf.add(layer3_1, layer_3_2)
    
    layer_final = tf.layers.conv2d_transpose(layer3_out, num_classes, 16, strides=(8,8), padding='same',
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    return layer_final
tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    label = tf.reshape(correct_label, (-1, num_classes))
    
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    cross_entropy_loss = \
        tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label)) + \
        tf.reduce_sum(reg_losses)

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss=cross_entropy_loss, global_step=tf.train.get_global_step())

    return logits, train_op, cross_entropy_loss


tests.test_optimize(optimize)

Tests Passed
Tests Passed
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Tests Passed


In [3]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    sess.run(tf.global_variables_initializer())
    
    print("Training..." + "\n")
    print()
    # TODO: Implement function
    for epoch in range(epochs):
        print("EPOCH {} ...".format(epoch+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict={
                input_image: image, correct_label: label,
                keep_prob: 0.5, learning_rate: 0.0009
            })

            print("Loss: = {:.3f}".format(loss))
        print()

#tests.test_train_nn(train_nn)

In [4]:
# Import everything needed to edit/save/watch video clips# Import 
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
import scipy
import numpy as np

clip = VideoFileClip('driving.mp4')

# TODO: Complete this function.
# The input is an NumPy array.
# The output should also be a NumPy array.
def pipeline(img):
    image_shape = (160, 576)

    draw_img = Image.fromarray(img)
    image = scipy.misc.imresize(draw_img, image_shape)

    im_softmax = sess.run(
            [tf.nn.softmax(logits)],
            feed_dict={keep_prob: 1.0, image_input: [image]})
    im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    mask = scipy.misc.toimage(mask, mode="RGBA")
    street_im = scipy.misc.toimage(image)
    street_im.paste(mask, box=None, mask=mask)

    return np.array(street_im)

In [5]:
num_classes = 2
image_shape = (160, 576)
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)
epochs = 50
batch_size = 5
use_saved_model = False
    
def run():
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        if not use_saved_model:
            # Path to vgg model
            vgg_path = os.path.join(data_dir, 'vgg')
            # Create function to get batches
            get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

            # OPTIONAL: Augment Images for better results
            #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

            # TODO: Build NN using load_vgg, layers, and optimize function
            input_image, keep_prob, vgg_layer3_out_tensor, vgg_layer4_out_tensor, vgg_layer7_out_tensor = \
                load_vgg(sess, vgg_path)
            layer_output = layers(vgg_layer3_out_tensor, vgg_layer4_out_tensor, vgg_layer7_out_tensor, num_classes)
            #print(tf.trainable_variables())

            correct_label = tf.placeholder("float", [None, None, None, num_classes])
            learning_rate = tf.placeholder("float", None)

            logits, train_op, cross_entropy_loss = optimize(layer_output, correct_label, learning_rate, num_classes)
        
            # TODO: Train NN using the train_nn function
            train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                     correct_label, keep_prob, learning_rate)

            # TODO: Save inference data using helper.save_inference_samples
            helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
            
            # Save model
            saver = tf.train.Saver()
            save_path = os.path.join(runs_dir, 'model.ckpt')
            saver.save(sess, save_path)
            print('Saved at : {}'.format(save_path))
        
        else:
            # OPTIONAL: Apply the trained model to a video
            saver = tf.train.import_meta_graph(os.path.join(runs_dir, 'model.ckpt.meta'))
            save_path = os.path.join(runs_dir, 'model.ckpt')
            saver.restore(sess, save_path)
            print('Restored from : {}'.format(save_path))
            
            graph = tf.get_default_graph()
            image_input = graph.get_operation_by_name("image_input")
            keep_prob = graph.get_operation_by_name("keep_prob")
            logits = graph.get_operation_by_name("logits")
            
            helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)
            
            #new_clip = clip.fl_image(pipeline)
            # write to file
            #new_clip.write_videofile('result.mp4')
            
            


if __name__ == '__main__':
    print("Starting...")
    run()
    print("Done!")

Tests Passed
Starting...
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Training...


EPOCH 1 ...
Loss: = 1.504
Loss: = 1.336
Loss: = 1.197
Loss: = 0.830
Loss: = 0.740
Loss: = 0.702
Loss: = 0.694
Loss: = 0.697
Loss: = 0.684
Loss: = 0.660
Loss: = 0.637
Loss: = 0.584
Loss: = 0.514
Loss: = 0.531
Loss: = 0.483
Loss: = 0.708
Loss: = 0.417
Loss: = 0.509
Loss: = 0.402
Loss: = 0.449
Loss: = 0.399
Loss: = 0.373
Loss: = 0.450
Loss: = 0.431
Loss: = 0.375
Loss: = 0.401
Loss: = 0.397
Loss: = 0.359
Loss: = 0.341
Loss: = 0.359
Loss: = 0.371
Loss: = 0.353
Loss: = 0.303
Loss: = 0.245
Loss: = 0.256
Loss: = 0.278
Loss: = 0.301
Loss: = 0.259
Loss: = 0.268
Loss: = 0.252
Loss: = 0.293
Loss: = 0.249
Loss: = 0.250
Loss: = 0.197
Loss: = 0.183
Loss: = 0.250
Loss: = 0.278
Loss: = 0.187
Loss: = 0.181
Loss: = 0.205
Loss: = 0.429
Loss: = 0.239
Loss: = 0.255
Loss: = 0.279
Loss: = 0.276
Loss: = 0.313
Loss: = 0.253
Loss: = 0.246

EPOCH 2 ...
Loss: = 0.264
Loss: = 0.277
Loss: = 0.264
Loss: 

Loss: = 0.080
Loss: = 0.088
Loss: = 0.089
Loss: = 0.090
Loss: = 0.076
Loss: = 0.100
Loss: = 0.081
Loss: = 0.086
Loss: = 0.075
Loss: = 0.073
Loss: = 0.052
Loss: = 0.041

EPOCH 11 ...
Loss: = 0.074
Loss: = 0.111
Loss: = 0.066
Loss: = 0.076
Loss: = 0.060
Loss: = 0.061
Loss: = 0.074
Loss: = 0.092
Loss: = 0.100
Loss: = 0.060
Loss: = 0.043
Loss: = 0.057
Loss: = 0.077
Loss: = 0.094
Loss: = 0.081
Loss: = 0.064
Loss: = 0.091
Loss: = 0.056
Loss: = 0.062
Loss: = 0.090
Loss: = 0.070
Loss: = 0.087
Loss: = 0.067
Loss: = 0.055
Loss: = 0.112
Loss: = 0.098
Loss: = 0.071
Loss: = 0.077
Loss: = 0.069
Loss: = 0.117
Loss: = 0.065
Loss: = 0.063
Loss: = 0.075
Loss: = 0.077
Loss: = 0.042
Loss: = 0.093
Loss: = 0.112
Loss: = 0.045
Loss: = 0.075
Loss: = 0.126
Loss: = 0.085
Loss: = 0.098
Loss: = 0.127
Loss: = 0.077
Loss: = 0.075
Loss: = 0.085
Loss: = 0.082
Loss: = 0.080
Loss: = 0.060
Loss: = 0.091
Loss: = 0.052
Loss: = 0.060
Loss: = 0.103
Loss: = 0.075
Loss: = 0.061
Loss: = 0.094
Loss: = 0.097
Loss: = 0.049

EPOCH

Loss: = 0.087
Loss: = 0.080
Loss: = 0.053
Loss: = 0.051
Loss: = 0.045
Loss: = 0.068
Loss: = 0.062
Loss: = 0.043
Loss: = 0.044
Loss: = 0.090
Loss: = 0.050
Loss: = 0.092
Loss: = 0.059
Loss: = 0.069
Loss: = 0.077
Loss: = 0.050

EPOCH 21 ...
Loss: = 0.054
Loss: = 0.047
Loss: = 0.051
Loss: = 0.117
Loss: = 0.102
Loss: = 0.062
Loss: = 0.079
Loss: = 0.072
Loss: = 0.077
Loss: = 0.082
Loss: = 0.089
Loss: = 0.066
Loss: = 0.060
Loss: = 0.066
Loss: = 0.037
Loss: = 0.049
Loss: = 0.049
Loss: = 0.067
Loss: = 0.052
Loss: = 0.063
Loss: = 0.047
Loss: = 0.054
Loss: = 0.073
Loss: = 0.061
Loss: = 0.111
Loss: = 0.055
Loss: = 0.056
Loss: = 0.053
Loss: = 0.065
Loss: = 0.026
Loss: = 0.075
Loss: = 0.058
Loss: = 0.053
Loss: = 0.052
Loss: = 0.060
Loss: = 0.061
Loss: = 0.046
Loss: = 0.049
Loss: = 0.048
Loss: = 0.039
Loss: = 0.048
Loss: = 0.066
Loss: = 0.090
Loss: = 0.051
Loss: = 0.084
Loss: = 0.050
Loss: = 0.076
Loss: = 0.049
Loss: = 0.066
Loss: = 0.068
Loss: = 0.075
Loss: = 0.076
Loss: = 0.047
Loss: = 0.060
Loss: 

Loss: = 0.053
Loss: = 0.037
Loss: = 0.032
Loss: = 0.040
Loss: = 0.045
Loss: = 0.060
Loss: = 0.041
Loss: = 0.027
Loss: = 0.038
Loss: = 0.030
Loss: = 0.043
Loss: = 0.044
Loss: = 0.051
Loss: = 0.028
Loss: = 0.041
Loss: = 0.033
Loss: = 0.038
Loss: = 0.025
Loss: = 0.034
Loss: = 0.059

EPOCH 31 ...
Loss: = 0.040
Loss: = 0.040
Loss: = 0.044
Loss: = 0.050
Loss: = 0.022
Loss: = 0.045
Loss: = 0.034
Loss: = 0.042
Loss: = 0.025
Loss: = 0.041
Loss: = 0.035
Loss: = 0.033
Loss: = 0.030
Loss: = 0.055
Loss: = 0.045
Loss: = 0.043
Loss: = 0.033
Loss: = 0.042
Loss: = 0.037
Loss: = 0.027
Loss: = 0.051
Loss: = 0.030
Loss: = 0.059
Loss: = 0.031
Loss: = 0.031
Loss: = 0.051
Loss: = 0.055
Loss: = 0.044
Loss: = 0.052
Loss: = 0.051
Loss: = 0.034
Loss: = 0.039
Loss: = 0.070
Loss: = 0.056
Loss: = 0.063
Loss: = 0.042
Loss: = 0.026
Loss: = 0.062
Loss: = 0.101
Loss: = 0.050
Loss: = 0.057
Loss: = 0.039
Loss: = 0.075
Loss: = 0.052
Loss: = 0.049
Loss: = 0.042
Loss: = 0.044
Loss: = 0.062
Loss: = 0.037
Loss: = 0.070
Loss: 

Loss: = 0.035
Loss: = 0.025
Loss: = 0.042
Loss: = 0.059
Loss: = 0.041
Loss: = 0.044
Loss: = 0.039
Loss: = 0.046
Loss: = 0.034
Loss: = 0.030
Loss: = 0.040
Loss: = 0.039
Loss: = 0.039
Loss: = 0.037
Loss: = 0.044
Loss: = 0.049
Loss: = 0.030
Loss: = 0.030
Loss: = 0.059
Loss: = 0.051
Loss: = 0.046
Loss: = 0.037
Loss: = 0.037
Loss: = 0.058

EPOCH 41 ...
Loss: = 0.046
Loss: = 0.028
Loss: = 0.041
Loss: = 0.043
Loss: = 0.045
Loss: = 0.026
Loss: = 0.047
Loss: = 0.024
Loss: = 0.028
Loss: = 0.028
Loss: = 0.051
Loss: = 0.052
Loss: = 0.035
Loss: = 0.046
Loss: = 0.034
Loss: = 0.051
Loss: = 0.047
Loss: = 0.035
Loss: = 0.041
Loss: = 0.030
Loss: = 0.056
Loss: = 0.033
Loss: = 0.032
Loss: = 0.044
Loss: = 0.038
Loss: = 0.055
Loss: = 0.035
Loss: = 0.034
Loss: = 0.037
Loss: = 0.027
Loss: = 0.032
Loss: = 0.044
Loss: = 0.024
Loss: = 0.032
Loss: = 0.033
Loss: = 0.030
Loss: = 0.038
Loss: = 0.043
Loss: = 0.035
Loss: = 0.035
Loss: = 0.058
Loss: = 0.031
Loss: = 0.041
Loss: = 0.043
Loss: = 0.051
Loss: = 0.044
Loss: 

Loss: = 0.033
Loss: = 0.039
Loss: = 0.029
Loss: = 0.030
Loss: = 0.034
Loss: = 0.027
Loss: = 0.033
Loss: = 0.036
Loss: = 0.024
Loss: = 0.026
Loss: = 0.044
Loss: = 0.030
Loss: = 0.041
Loss: = 0.035
Loss: = 0.035
Loss: = 0.021
Loss: = 0.029
Loss: = 0.033
Loss: = 0.034
Loss: = 0.025
Loss: = 0.034
Loss: = 0.041
Loss: = 0.040
Loss: = 0.041
Loss: = 0.043
Loss: = 0.026
Loss: = 0.023
Loss: = 0.038

Training Finished. Saving test images to: ./runs/1524959169.490551
Saved at : ./runs/model.ckpt
Done!
